In [2]:
import pandas as pd

In [3]:
assignmnet1_emission = pd.read_csv("../out/emissions_assignment1.csv")
assignmnet1_emission

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2024-05-02T10:17:12,assignment 1,02872c40-cdda-48af-8bc3-5366bf628787,327.659156,0.003016,0.000009,42.5,0.0,141.200752,0.003868,...,64,Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz,NaN,NaN,12.5775,55.684,376.535339,machine,N,1.0


In [4]:
assignmnet1_emission_subtask = pd.read_csv("../out/emissions_base_cf8786c8-77d3-4a32-9d56-8fc20a39c656.csv")
assignmnet1_emission_subtask

,task_name,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,codecarbon_version,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud
0,load spacy model,2024-05-02T14:18:54,assignment 1,cf8786c8-77d3-4a32-9d56-8fc20a39c656,1.608494,0.000015,0.000009,42.5,0.0,141.200752,...,2.3.5,64,Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz,NaN,NaN,12.5775,55.684,376.535339,machine,N
1,"load data, process text, and save results",2024-05-02T14:22:22,assignment 1,cf8786c8-77d3-4a32-9d56-8fc20a39c656,208.238420,0.001917,0.000009,42.5,0.0,141.200752,...,2.3.5,64,Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz,NaN,NaN,12.5775,55.684,376.535339,machine,N
2,plot results,2024-05-02T14:22:23,assignment 1,cf8786c8-77d3-4a32-9d56-8fc20a39c656,1.196526,0.000011,0.000009,42.5,0.0,141.200752,...,2.3.5,64,Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz,NaN,NaN,12.5775,55.684,376.535339,machine,N


In [3]:
assignmnet3_emission = pd.read_csv("../out/emissions_assignment3.csv")
assignmnet3_emission

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2024-05-02T13:01:31,assignment 3,67ce7b5c-8235-4cce-8a12-3de6f9c92b49,0.004226,0.000147,0.034873,42.5,0.0,141.200752,0.000189,...,64,Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz,NaN,NaN,12.5775,55.684,376.535339,machine,N,1.0
